# IS_AGGRESIEVE_POSE with OPENPOSE

### Concept Overview 
1. Pose Estimation: 
    * Pose estimation invovles detecting key points (joints) on the human body in an image, such as the head, shoulders, elbows and knees 
    * Tools like OpenpOse ouput these key points along with their coordinates 
2. Defining "Aggressive" Poses: 
    * Raised hands holding a weapon 
    * Swining arms (indicative of hitting or threatening behavior)
    * a stance suggesting an attacj (leaning forward, legs wide apart )
3. Analyzing Pose Data: 
    * key points can be analyzed to infer these beahviors. For example:
        - Raised hands: The y-coordinate of hand joints is higher than the shoulder joints
        - Leaning forward: the neck joints is fat ahead of the jip joint on the x-axis

### Steps to Implement 
1. Understand the pose Data Format: 
    * Pose estimation tools like OpenPose output data in teh form of a list of key points 
    * Each key point contains the (x,y) coordinates and a confidence score 

In [1]:
pose_data = [
    {"keypoint": "nose", "x": 320, "y": 200, "confidence": 0.95},
    {"keypoint": "left_shoulder", "x": 300, "y": 250, "confidence": 0.90},
    {"keypoint": "right_shoulder", "x": 340, "y": 250, "confidence": 0.88},
    {"keypoint": "left_hand", "x": 280, "y": 150, "confidence": 0.85},
    {"keypoint": "right_hand", "x": 360, "y": 150, "confidence": 0.87},

]

2. Define Aggression Rule 
    * Rasie hands: The y-coordinate of hands is significantly higher than the shoulders 
    * Learning forward: The x-coordinate difference between the neck and hips is lareg 
    * Weapon stance: If weapons are detced near the hands (optional step combining object detection)
    

3. Exampel Implementation: 

In [2]:
def get_keypoint(pose_data, key):
    for point in pose_data:
        if point["keypoint"] == key:
            return point
    return None

In [3]:
# Hilfsfunktion zur Erkennung aggressiver Posen
def is_aggressive_pose(pose_data):
    left_hand = get_keypoint(pose_data, "left_hand")
    right_hand = get_keypoint(pose_data, "right_hand")
    left_shoulder = get_keypoint(pose_data, "left_shoulder")
    right_shoulder = get_keypoint(pose_data, "right_shoulder")
    neck = get_keypoint(pose_data, "Neck")
    mid_hip = get_keypoint(pose_data, "mid_hip")
    
    # Check if hands are raised 
    if left_hand and left_shoulder and left_hand["y"] < left_shoulder["y"]:
        return True
    if right_hand and right_shoulder and right_hand["y"] < right_shoulder["y"]:
        return True
    
    # Check if leaning forward 
    if neck and mid_hip and abs(neck["x"]- mid_hip["x"])>50: #Threshold for leaning forwward
        return True 
    return True

In [4]:
sample_pose_data = [
    {"keypoint": "left_hand", "x": 280, "y": 150, "confidence": 0.85},
    {"keypoint": "left_shoulder", "x": 300, "y": 250, "confidence": 0.90},
    {"keypoint": "neck", "x": 320, "y": 200, "confidence": 0.95},
    {"keypoint": "mid_hip", "x": 300, "y": 300, "confidence": 0.88},
]
print(is_aggressive_pose(sample_pose_data))  # Output: True (hands raised)


True


### Extension 
* Add weapon detection: Combine pose data with YOLOv8 results to check if detected weapons are near hands.
* Consider multiple persons: Iterate through pose data for each detected person in the image.

# Actively Shooting 

### Key Pose Indicators for Shooting: 
1. Weapon in Hand 
    * The detected weapon (from YOLOv8 or similar tools) is clsoe to the hand key points (left or right hand)
    * => the bounding box center of the weapon overlaps with coordinates of the hand 
2. Arm extended in Shooting Direction 
    * One or both arms are extended toward another person, indicating amining 
    * This can be detected if the hand joints is far form the shoulder joint but roughly aligend along a straight line with the shoulder and elbow joints.
3. Body Orientation 
    * The body is directed toward another person. This is inferred by the alignment of key points like shoulders and hips 
    * If teh distance between the shooters torso key points (neck, shoulders, hips) and the target is small in the x-axsi or y-axis 
4. Aggresive Stance  
    * a shooting posture often involves a slight forward lean for balance 
    * the neck joint is slightly ahead of the hip joint on the x-axis

### Implementation Steps 


In [ ]:
# 1. Weapon Proximity to Hand 
def is_weapon_in_hand(weapon_box, hand_keypoint):
    # weapon_box = {"x_min": x_min, "x_max": x_max, "y_min": y_min, "y_max": y_max}
    # hand_keypoint = {"x": x, "y": y, "confidence": confidence}
    weapon_center_x = (weapon_box["x_min"]+ weapon_box["x_max"])/2
    weapon_center_y = (weapon_box["y_min"]+ weapon_box["y_max"])/2
    
    hand_x, hand_y = hand_keypoint["x"], hand_keypoint["y"]
    
    # Check if the hand is close to the weapon
    distance = ((weapon_center_x - hand_x)**2 + (weapon_center_y - hand_y)**2)**0.5
    return distance < 50  # Adjust threshold based on dataset

In [ ]:
# 2. Arm Extension 
def is_arm_extedned(shoulder, elbow, hand):
    # Check if the arm is extended ina roughly straight line 
    def distance(p1, p2):
        return ((p1["x"] - p2["x"])**2 + (p1["y"] - p2["y"])**2)**0.5
    
    shoulder_to_elbow = distance(shoulder, elbow)
    elbow_to_hand = distance(elbow, hand)
    shoulder_to_hand = distance(shoulder, hand)
    
    # If the sum of the two shorter segments equals the long segment, it’s a straight line
    return abs((shoulder_to_elbow + elbow_to_hand) - shoulder_to_hand) < 10  # Adjust tolerance


In [ ]:
# 3. Direction of Aim 
def is_aiming_at_target(shooter_pose, target_pose):
    shooter_hand = shooter_pose["right_hand"]
    target_torso = target_pose["neck"]
    
    dx = target_torso["x"]-shooter_hand["x"]
    dy = target_torso["y"]-shooter_hand["y"]
    angle = abs(math.atan2(dy, dx)* 180/math.pi)
    # Check if angle is within a typical aiming range (e.g., 0°–45° horizontally)
    return 0 <= angle <= 45
    